In [15]:
!pip install requests_cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00


In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests
import requests_cache


In [18]:
session = requests_cache.CachedSession()
session.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'

# Patch yfinance to use this session
yf.shared._requests = session  # this is the internal request patch

# Download data using yf.download()
ticker = 'AAPL'
df = yf.download(ticker, period='6mo', interval='1d', progress=False)

# Clean
df = df[['Close']].copy()
print(df.head())


1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Empty DataFrame
Columns: [(Close, AAPL)]
Index: []


In [ ]:

# --- Step 2: Calculate RSI (14-day) ---
delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.rolling(window=14, min_periods=14).mean()
avg_loss = loss.rolling(window=14, min_periods=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# --- Step 3: Plot 1 - Price + RSI Chart ---
price_fig = go.Figure()
price_fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='AAPL Price'))

price_fig.update_layout(
    title="📈 AAPL Price",
    template='plotly_white'
)

rsi_fig = go.Figure()
rsi_fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='blue')))
rsi_fig.add_shape(type='line', x0=df.index[0], x1=df.index[-1], y0=70, y1=70,
                  line=dict(color='red', dash='dash'), name='Overbought')
rsi_fig.add_shape(type='line', x0=df.index[0], x1=df.index[-1], y0=30, y1=30,
                  line=dict(color='green', dash='dash'), name='Oversold')
rsi_fig.update_layout(
    title="📊 RSI Indicator (14-day)",
    yaxis=dict(title="RSI"),
    template='plotly_white'
)

# --- Step 4: Alpha Decay Calculation ---
df['Returns'] = df['Close'].pct_change()
df['Signal'] = df['RSI'] < 30  # Buy when RSI < 30
N = 10  # look 10 days ahead
decay_matrix = []

trigger_dates = df[df['Signal']].index

for date in trigger_dates:
    idx = df.index.get_loc(date)
    if idx + N < len(df):
        future_returns = df['Returns'].iloc[idx+1:idx+N+1].values
        decay_matrix.append(future_returns)

if decay_matrix:
    decay_df = pd.DataFrame(decay_matrix)
    decay_avg = decay_df.mean()
else:
    decay_avg = pd.Series([0]*N)

decay_fig = go.Figure()
decay_fig.add_trace(go.Bar(
    x=[f"T+{i+1}" for i in range(N)],
    y=decay_avg,
    name='Avg Return'
))
decay_fig.update_layout(
    title="🔬 Alpha Decay After RSI < 30",
    yaxis_title="Avg Return",
    template='plotly_white'
)

# --- Step 5: RSI-Based Strategy Cumulative Return ---
df['Position'] = 0
holding_period = 3
for date in trigger_dates:
    idx = df.index.get_loc(date)
    df.loc[df.index[idx+1:idx+1+holding_period], 'Position'] = 1

df['Strategy'] = df['Position'] * df['Returns']
df['Strategy_Cum'] = (1 + df['Strategy'].fillna(0)).cumprod()
df['BuyHold_Cum'] = (1 + df['Returns'].fillna(0)).cumprod()

cum_fig = go.Figure()
cum_fig.add_trace(go.Scatter(x=df.index, y=df['BuyHold_Cum'], name='Buy & Hold'))
cum_fig.add_trace(go.Scatter(x=df.index, y=df['Strategy_Cum'], name='RSI Strategy'))
cum_fig.update_layout(
    title="💹 Cumulative Returns: RSI < 30 vs Buy & Hold",
    yaxis_title="Growth of $1",
    template='plotly_white'
)

# --- Step 6: Show or Save ---
# Show in Jupyter or script
price_fig.show()
rsi_fig.show()
decay_fig.show()
cum_fig.show()

# Optionally save images
# price_fig.write_image("1_price.png")
# rsi_fig.write_image("2_rsi.png")
# decay_fig.write_image("3_decay.png")
# cum_fig.write_image("4_cum.png")